In [ ]:
import numpy as np
import pandas as pd
import openpyxl
from conditionalAssetPricingLogMarginalLikelihoodTauClass import Model

# === Step 1: Setup ===
homeDir = r'C:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\speciale_repo\Complemetary Code Files for Submission\\'
dataDir = homeDir + r'Data\\'

directory_name_prefix = homeDir + 'conditionalTauDMN0F14M13s'

# === Step 2: Choose number of predictors to use ===
n_predictors_to_use = 2  # 🔧 Set this to anything between 1 and 13
assert 1 <= n_predictors_to_use <= 13, "You must choose between 1 and 13 predictors"

# === Step 3: Load test assets (returns) ===
returns = pd.read_excel(dataDir + 'Returns.xlsx', sheet_name="Industry_returns")
returns = returns.iloc[:-1]  # remove first row

returns = returns.drop(columns=['Grand Total'])  # remove Date column
for col in returns.columns:
    returns[col] = returns[col] * 100
# === Step 4: Load and clean factors ===
factors = pd.read_csv(dataDir + 'factors-20.csv')
factors = factors.drop(columns=['MKTRF','SMB*','MKT','CON','IA', 'ROE', 'ME'])  # remove redundant/duplicated
factorsNames = factors.columns.drop('Date')
for col in factors.columns:
   if col != 'Date':
        factors[col] = factors[col] * 100



In [ ]:
print(returns.columns.tolist())

In [ ]:
returns.drop(columns="Row Labels", inplace=True)
returns

In [ ]:

# === Step 5: Load and clean predictors ===
predictors = pd.read_csv(dataDir + 'Z - 197706.csv')
if 'Unnamed: 0' in predictors.columns:
    predictors = predictors.drop(columns='Unnamed: 0')

# Define full predictor index list
full_predictor_list = predictors.columns.drop('Date')
significantPredictors = np.arange(n_predictors_to_use)
predictorsNames = full_predictor_list[significantPredictors]

# Subset actual predictor data
predictors = predictors[['Date'] + predictorsNames.tolist()]


In [ ]:
print(predictors.describe().T[['mean', 'std']])

In [ ]:
factors

In [ ]:

# === Step 6: Truncate all to shared sample size and index ===
start_date = '1997-01-01'
end_date = '2006-12-01'
date_index = pd.date_range(start=start_date, end=end_date, freq='MS')
n = len(date_index)

returns = returns.iloc[:n].copy()

#returns_sim = returns_sim.iloc[:n].copy()

factors = factors.iloc[:n].copy()
predictors = predictors.iloc[:n].copy()
# === Convert decimal returns to percent returns ===

returns.index = date_index
factors.index = date_index
predictors.index = date_index

# === Step 7: Reset index and rename for model compatibility ===
returns_reset = returns.reset_index().rename(columns={'index': 'Date'})
#returns_reset.drop(columns='Row Labels', inplace=True) 

#returns_sim_reset = returns_sim.reset_index().rename(columns={'index': 'Date'})

factors_reset = factors.reset_index().rename(columns={'index': 'Date'})
predictors_reset = predictors.reset_index().rename(columns={'index': 'Date'})


In [ ]:
returns_reset

In [ ]:
returns

In [ ]:

# === Step 8: Define model parameters ===
Tau = 1.5
index_end_of_estimation = 118  # Half of length of returns

# === Step 9: Instantiate model ===
model = Model(rr=returns_reset,
              ff=factors_reset,
              zz=predictors_reset,
              significantPredictors=significantPredictors,
              Tau=Tau,
              indexEndOfEstimation=index_end_of_estimation,
              key_demean_predictors=True)

# === Step 10: Run conditional BMA expected return calculation ===
nModelsMax = pow(2, model.KMax + model.MMax)
uniform_probs = np.ones(nModelsMax) / nModelsMax
models_probabilities = np.concatenate([uniform_probs, uniform_probs])

result = model.conditionalAssetPricingOOSTauNumba(
    models_probabilities=models_probabilities,
    nModelsInPrediction=0  # Use full model space
)




In [ ]:
result[0]